# अध्याय 7 पाठ विस्तार

<div class='toc'>
<उल क्लास='टोक-आइटम'>
<li><span><a href='#一 परिचय' data-toc-modified-id='1. परिचय'>1. परिचय</a></span></li>
<ली>
<span><a href='#二अनुकूलित ग्राहक मेल' data-toc-modified-id='二、अनुकूलित ग्राहक मेल'>二、अनुकूलित ग्राहक मेल</a></span>
</li>
<li><span><a href=”#三义化 तापमान विभाग” data-toc-modified-id=”三、यिनहुआ तापमान विभाग”>三、यिनहुआ तापमान विभाग</a></span>
</li>
</ul>
</div>

## 1 परिचय

एक्सटेंशन एक बड़े भाषा मॉडल में छोटे पाठ (जैसे निर्देशों का एक सेट या विषयों की एक सूची) के इनपुट हैं, जो मॉडल को लंबे पाठ (जैसे एक निश्चित विषय पर आधारित ईमेल या निबंध) उत्पन्न करने की अनुमति देते हैं दोधारी तलवार, एक बड़े भाषा मॉडल को विचार-मंथन भागीदार के रूप में उपयोग करने जैसे लाभों के साथ, लेकिन कुछ समस्याएं भी हैं, जैसे कि कोई इसका उपयोग बहुत अधिक स्पैम उत्पन्न करने के लिए कर सकता है, इसलिए, जब आप बड़े भाषा मॉड की इन सुविधाओं का उपयोग करते हैंदोस्तों, कृपया इनका उपयोग केवल **जिम्मेदार** और **लोगों की मदद** करने के तरीके से करें।

इस अध्याय में, आप सीखेंगे कि ओपनएआई एपीआई के आधार पर *प्रत्येक ग्राहक समीक्षा के लिए अनुकूलित* ग्राहक सेवा ईमेल कैसे तैयार करें। हम तापमान नामक मॉडल के एक अन्य इनपुट पैरामीटर का भी लाभ उठाएंगे, जो आपको डिग्री और विविधता को अलग करने की अनुमति देता है मॉडल प्रतिक्रिया में अन्वेषण का।

पिछले अध्यायों की तरह, आपको ऐसे वातावरण को कॉन्फ़िगर करने के लिए समान कोड की आवश्यकता है जो OpenAI API का उपयोग कर सके

In [1]:
import openai
# तृतीय-पक्ष लाइब्रेरी आयात करें

openai.api_key = "sk-..."
# API_KEY सेट करें, कृपया इसे अपनी API_KEY से बदलें


In [9]:
# एक फ़ंक्शन जो ओपनएआई इंटरफ़ेस को प्रॉम्प्ट के एक पैरामीटर के साथ इनकैप्सुलेट करता है और संबंधित परिणाम देता है
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    temperature: 温度系数
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 模型输出的温度系数，控制输出的随机程度
    )
# OpenAI के चैटकंप्लीशन इंटरफ़ेस पर कॉल करें
    return response.choices[0].message["content"]


## 2. ग्राहक ईमेल अनुकूलित करें

हम ग्राहक समीक्षाओं और भावनाओं के आधार पर लक्षित ऑटो-रिप्लाई ईमेल लिखेंगे, इसलिए, हम ग्राहक समीक्षाओं और भावनाओं के आधार पर लक्षित प्रतिक्रियाएं उत्पन्न करने के लिए एलएलएम का उपयोग करेंगे, यानी ग्राहक समीक्षाओं और समीक्षा भावनाओं के आधार पर अनुकूलित ईमेल उत्पन्न करेंगे।

हम पहले एक उदाहरण देते हैं, जिसमें एक समीक्षा और उससे संबंधित भावना शामिल है।

In [4]:
# "अनुमान" पर पाठ से भावना दी गई,
# और मूल ग्राहक संदेश, ईमेल को अनुकूलित करें
sentiment = "negative"

# ब्लेंडर की समीक्षा
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [11]:
# हम अनुमान अध्याय में किसी टिप्पणी की भावना का मूल्यांकन करना सीख सकते हैं
sentiment = "negative"

#एक उत्पाद समीक्षा
review = f"""
他们在11月份的季节性销售期间以约49美元的价格出售17件套装，折扣约为一半。\
但由于某些原因（可能是价格欺诈），到了12月第二周，同样的套装价格全都涨到了70美元到89美元不等。\
11件套装的价格也上涨了大约10美元左右。\
虽然外观看起来还可以，但基座上锁定刀片的部分看起来不如几年前的早期版本那么好。\
不过我打算非常温柔地使用它，例如，\
我会先在搅拌机中将像豆子、冰、米饭等硬物研磨，然后再制成所需的份量，\
切换到打蛋器制作更细的面粉，或者在制作冰沙时先使用交叉切割刀片，然后使用平面刀片制作更细/不粘的效果。\
制作冰沙时，特别提示：\
将水果和蔬菜切碎并冷冻（如果使用菠菜，则轻轻煮软菠菜，然后冷冻直到使用；\
如果制作果酱，则使用小到中号的食品处理器），这样可以避免在制作冰沙时添加太多冰块。\
大约一年后，电机发出奇怪的噪音，我打电话给客服，但保修已经过期了，所以我不得不再买一个。\
总的来说，这些产品的总体质量已经下降，因此它们依靠品牌认可和消费者忠诚度来维持销售。\
货物在两天内到达。
"""

अब जब हमने अनुमान पाठ्यक्रम में सीखी गई विधियों का उपयोग करके एक ब्लेंडर के बारे में ग्राहक समीक्षा की भावना निकाल ली है, तो हम भावना के आधार पर अपनी प्रतिक्रिया तैयार करेंगे।

निम्नलिखित संकेत को एक उदाहरण के रूप में लें: मान लीजिए कि आप एक ग्राहक सेवा एआई सहायक हैं और आपका कार्य ग्राहक के लिए तीन बैकटिक्स द्वारा अलग किए गए ईमेल उत्तर के आधार पर, समीक्षा के लिए ग्राहक को धन्यवाद देने के लिए एक उत्तर तैयार करना है।

In [5]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt)
print(response)

Dear Valued Customer,

Thank you for taking the time to leave a review about our product. We are sorry to hear that you experienced an increase in price and that the quality of the product did not meet your expectations. We apologize for any inconvenience this may have caused you.

We would like to assure you that we take all feedback seriously and we will be sure to pass your comments along to our team. If you have any further concerns, please do not hesitate to reach out to our customer service team for assistance.

Thank you again for your review and for choosing our product. We hope to have the opportunity to serve you better in the future.

Best regards,

AI customer agent


In [6]:
prompt = f"""
你是一位客户服务的AI助手。
你的任务是给一位重要客户发送邮件回复。
根据客户通过“```”分隔的评价，生成回复以感谢客户的评价。提醒模型使用评价中的具体细节
用简明而专业的语气写信。
作为“AI客户代理”签署电子邮件。
客户评论：
```{review}```
评论情感：{sentiment}
"""
response = get_completion(prompt)
print(response)

尊敬的客户，

非常感谢您对我们产品的评价。我们非常抱歉您在购买过程中遇到了价格上涨的问题。我们一直致力于为客户提供最优惠的价格，但由于市场波动，价格可能会有所变化。我们深表歉意，如果您需要任何帮助，请随时联系我们的客户服务团队。

我们非常感谢您对我们产品的详细评价和使用技巧。我们将会把您的反馈传达给我们的产品团队，以便改进我们的产品质量和性能。

再次感谢您对我们的支持和反馈。如果您需要任何帮助或有任何疑问，请随时联系我们的客户服务团队。

祝您一切顺利！

AI客户代理


## 3. तापमान गुणांक का परिचय

इसके बाद, हम "तापमान" नामक भाषा मॉडल के एक पैरामीटर का उपयोग करेंगे जो हमें मॉडल की प्रतिक्रियाओं की विविधता को बदलने की अनुमति देगा। आप तापमान को मॉडल की खोज या यादृच्छिकता की डिग्री के रूप में सोच सकते हैं।

उदाहरण के लिए, किसी विशेष वाक्यांश में, "मेरा पसंदीदा भोजन" के लिए सबसे संभावित अगला शब्द "पिज्जा" है, उसके बाद "सुशी" और "टैको" है, इसलिए शून्य के तापमान पर, मॉडल हमेशा सबसे संभावित अगला शब्द चुनेगा शब्द, जबकि हाय परअधिक तापमान पर यह कम संभावित शब्दों में से एक को भी चुनेगा, और इससे भी अधिक तापमान पर यह टैको को भी चुन सकता है, जिसकी संभावना केवल पांच गुना में से एक है, आप कल्पना कर सकते हैं कि जैसे-जैसे मॉडल अधिक शब्द उत्पन्न करना जारी रखेगा, अंतिम प्रतिक्रिया होगी। मेरा पसंदीदा भोजन पिज़्ज़ा है" पहली प्रतिक्रिया से भिन्न होगा "मेरा पसंदीदा भोजन टैको है"। जैसे-जैसे मॉडल जारी रहेगा, ये दोनों प्रतिक्रियाएं भी अधिक से अधिक भिन्न होती जाएंगी।

सामान्य तौर पर, ऐसे एप्लिकेशन बनाते समय जिनके लिए पूर्वानुमानित सम्मान की आवश्यकता होती हैऑनसेस, मैं **तापमान को शून्य पर सेट करने की सलाह देता हूं**। सभी पाठ्यक्रमों में, हम तापमान को शून्य पर सेट कर रहे हैं, और यदि आप एक विश्वसनीय और पूर्वानुमानित प्रणाली बनाने की कोशिश कर रहे हैं, तो मुझे लगता है कि आपको यह तापमान चुनना चाहिए मॉडल को अधिक रचनात्मक तरीकों से उपयोग करने का प्रयास करें, शायद विभिन्न आउटपुट की व्यापक रेंज के साथ, आप उच्च तापमान का उपयोग करना चाह सकते हैं।

उसी पत्र में, हम मॉडल को उपयोगकर्ता के पत्र से विवरण का उपयोग करने और तापमान निर्धारित करने की याद दिलाते हैं:

In [10]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt, temperature=0.7)
print(response)

Dear valued customer,

Thank you for taking the time to share your review with us. We are sorry to hear that you were disappointed with the prices of our products and the quality of our blender. We apologize for any inconvenience this may have caused you.

We value your feedback and would like to make things right for you. Please feel free to contact our customer service team so we can assist you with any concerns or issues you may have. We are committed to providing you with the best possible service and products.

Thank you again for your review and for being a loyal customer. We hope to have the opportunity to serve you better in the future.

Sincerely,
AI customer agent


In [12]:
prompt = f"""
你是一名客户服务的AI助手。
你的任务是给一位重要的客户发送邮件回复。
根据通过“```”分隔的客户电子邮件生成回复，以感谢客户的评价。
如果情感是积极的或中性的，感谢他们的评价。
如果情感是消极的，道歉并建议他们联系客户服务。
请确保使用评论中的具体细节。
以简明和专业的语气写信。
以“AI客户代理”的名义签署电子邮件。
客户评价：```{review}```
评论情感：{sentiment}
"""
response = get_completion(prompt, temperature=0.7)
print(response)

尊敬的客户，

非常感谢您对我们产品的评价。我们由衷地为您在购买过程中遇到的问题表示抱歉。我们确实在12月份的第二周调整了价格，但这是由于市场因素所致，并非价格欺诈。我们深刻意识到您对产品质量的担忧，我们将尽一切努力改进产品，以提供更好的体验。

我们非常感激您对我们产品的使用经验和制作技巧的分享。您的建议和反馈对我们非常重要，我们将以此为基础，进一步改进我们的产品。

如果您有任何疑问或需要进一步帮助，请随时联系我们的客户服务部门。我们将尽快回复您并提供帮助。

最后，请再次感谢您对我们产品的评价和选择。我们期待着未来与您的合作。

此致

敬礼

AI客户代理


शून्य के तापमान पर, आपको हर बार एक ही संकेत निष्पादित करने पर एक ही प्रतिक्रिया मिलनी चाहिए। 0.7 के तापमान पर, आपको हर बार एक अलग आउटपुट मिलता है।

तो आप देख सकते हैं कि यह हमें पहले मिले ईमेल से अलग है। इसे दोबारा निष्पादित करने पर आपको फिर से एक अलग ईमेल मिलेगा।

इसलिए मेरा सुझाव है कि आप स्वयं तापमान के साथ प्रयोग करके देखें कि आउटपुट कैसे बदलता है। संक्षेप में, उच्च तापमान पर, मॉडल का आउटपुट लगभग अधिक यादृच्छिक होता हैजी **अधिक ध्यान भटकाने वाला**, लेकिन शायद **अधिक रचनात्मक**, उच्च तापमान पर।